# XGBoost Regression Model

In [1]:
import kagglehub
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

# Regression Classifier.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Statsmodels.
import statsmodels.api as sm
import pandas as pd
import pickle

# XGBoost.
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV

/Users/nathanlonghurst/Desktop/ACME/vol3-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load dataset.
# Download latest version
path = kagglehub.dataset_download("lashagoch/life-expectancy-who-updated")
print("Path to dataset files:", path)

file = "Life-Expectancy-Data-Updated.csv"

# TODO:  Correct features and assign target.
data = pd.read_csv(path + "/" + file)
data.columns
features_to_drop = ['Infant_deaths', 'Under_five_deaths', 'Adult_mortality', 'Country', 'Region', 'Year']
X = data.drop(columns=features_to_drop + ['Life_expectancy'])
# X = data[['Alcohol_consumption', 'Hepatitis_B', 'Measles',
#        'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
#        'Population_mln', 'Thinness_ten_nineteen_years',
#        'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed']]

y = data['Life_expectancy']

# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Path to dataset files: /Users/nathanlonghurst/.cache/kagglehub/datasets/lashagoch/life-expectancy-who-updated/versions/1


In [3]:
# Construct XGBoost regression model.
model = XGBRegressor(
    n_estimators=100,       # Number of boosted trees.
    learning_rate=0.1,      # Shrinkage (eta).
    max_depth=6,            # Tree depth.
    subsample=1.0,          # Row sampling.
    colsample_bytree=1.0,   # Feature sampling.
    random_state=0
)

# Fit model and predict results.
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

mse  = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2   = r2_score(y_test, y_pred)

corrs_clean = data[X.columns.tolist() + ['Life_expectancy']].corr(numeric_only=True)['Life_expectancy'].sort_values(ascending=False)
print(f'corrs:  {corrs_clean}')

print(f'mse:  {mse}')
print(f'rmse:  {rmse}')
print(f'r2:  {r2}')

corrs:  Life_expectancy                1.000000
Schooling                      0.732484
Polio                          0.641217
Diphtheria                     0.627541
BMI                            0.598423
GDP_per_capita                 0.583090
Economy_status_Developed       0.523791
Measles                        0.490019
Hepatitis_B                    0.417804
Alcohol_consumption            0.399159
Population_mln                 0.026298
Thinness_five_nine_years      -0.458166
Thinness_ten_nineteen_years   -0.467824
Economy_status_Developing     -0.523791
Incidents_HIV                 -0.553027
Name: Life_expectancy, dtype: float64
mse:  1.2011250399609759
rmse:  1.0959585028462417
r2:  0.9855270801647514


In [ ]:
# GridSearch
param_grid = {
    'max_depth': [3, 4, 5, 6, 8], # max depth for a particular branch of tree
    'learning_rate': [0.01, 0.05, 0.1], # learning rate for fitting
    'subsample': [0.7, 0.85, 1.0], # percentage of data to subsample
    'colsample_bytree': [0.6, 0.8, 1.0], # percentage of features to sample for tree
    'n_estimators': [200, 400, 600] # number of boosting rounds
}

grid = GridSearchCV(
    XGBRegressor(random_state=42),
    param_grid,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid.fit(X_train, y_train)
print(grid.best_params_)
neg_mean_squared_error = grid.best_score_
print(grid.best_score_)

{'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 600, 'subsample': 0.7}
-0.9983217930692707


In [5]:
best_model = grid.best_estimator_
importances = best_model.feature_importances_
feature_importances = pd.Series(importances, index=X_train.columns)
feature_importances = feature_importances.sort_values(ascending=False)
print("order of feature importances")
print(feature_importances)

order of feature importances
Incidents_HIV                  0.562993
GDP_per_capita                 0.179126
Schooling                      0.088710
BMI                            0.037584
Economy_status_Developed       0.036522
Diphtheria                     0.019509
Thinness_five_nine_years       0.019026
Polio                          0.017947
Thinness_ten_nineteen_years    0.014524
Population_mln                 0.008939
Measles                        0.008751
Alcohol_consumption            0.003267
Hepatitis_B                    0.003102
Economy_status_Developing      0.000000
dtype: float32


In [6]:
# Save the model.
with open('XGBoost.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [7]:
# Fit model and predict results.
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [10]:
# Predictions.
y_test_pred = model.predict(X_test)

# Metrics.
test_mse = mean_squared_error(y_test, y_test_pred)

print(f"Train neg MSE: {neg_mean_squared_error}")
print(f'Test neg MSE:  {-test_mse}')

Train neg MSE: -0.9983217930692707
Test neg MSE:  -1.2011250399609759
